In [1]:
import numpy as np
import skfmm
import os
def loadswc(filepath):
    '''
    Load swc file as a N X 7 numpy array
    '''
    swc = []
    with open(filepath) as f:
        lines = f.read().split("\n")
        for l in lines:
            if not l.startswith('#'):
                cells = l.split(' ')
                if len(cells) ==7:
                    cells = [float(c) for c in cells]
                    swc.append(cells)
    return np.array(swc)

def loadtiff3d(filepath):
    """Load a tiff file into 3D numpy array"""

    import tifffile as tiff
    a = tiff.imread(filepath)

    stack = []
    for sample in a:
        stack.append(np.rot90(np.fliplr(np.flipud(sample))))
    out = np.dstack(stack)

    return out


def writetiff3d(filepath, block):
    import tifffile as tiff

    try:
        os.remove(filepath)
    except OSError:
        pass

    with tiff.TiffWriter(filepath, bigtiff=False) as tif:
        for z in range(block.shape[2]):
            saved_block = np.rot90(block[:, :, z])
            tif.save(saved_block.astype('uint8'), compress=0)

In [2]:
def crop(img):
    ind = np.argwhere(img>0)
    x = ind[:,0]
    y = ind[:,1]
    z = ind[:,2]
    xmin = max(x.min()-5, 0)
    xmax = min(x.max()+5, img.shape[0])
    ymin = max(y.min()-5, 0)
    ymax = max(y.min()+5, img.shape[1])
    zmin = max(z.min()-5, 0)
    zmax = max(z.min()+5, img.shape[2])
    
    return np.array([[xmin, xmax], [ymin, ymax], [zmin, zmax]])
    

In [3]:
'''
Regenerate tif file from swc and also apply distance transform.
'''
def swc2tif_dt(swc, img):
    import math

    shape = img.shape
    skimg = np.ones(shape)
    zeromask = np.ones(shape)

    # Add nodes the current swc to make sure there is
    # at least one node in each voxel on a branch
    idlist = swc[:, 0]
    extra_nodes = []
    extra_nodes_radius = []
    for i in range(swc.shape[0]):
        cnode = swc[i, 2:5]
        c_radius = swc[i, -2]
        pnode = swc[idlist == swc[i, 6], 2:5]
        if pnode.shape[0] != 0:
            p_radius = swc[idlist == swc[i, 6], -2][0]
            average_radius = int(c_radius+p_radius)/2

        dvec = pnode - cnode # [[x, y, z]]
        dvox = np.floor(np.linalg.norm(dvec)) # eculidean norm
        if dvox >= 1:
            uvec = dvec / (dvox + 1) # unit vector
            extra_nodes.extend(
                [cnode + uvec * i for i in range(1, int(dvox))])
            extra_nodes_radius.extend([average_radius for i in range(1, int(dvox))])

    # Deal with nodes in swc
    for i in range(swc.shape[0]):
        node = [math.floor(n) for n in swc[i, 2:5]]
        for j in range(3):
            if node[j] > shape[j]-1:
                node[j] = shape[j]-1
        r = int(swc[i, -2])
        skimg[node[0], node[1], node[2]] = 0
        zeromask[max(0,node[0]-r): min(node[0]+r, shape[0]), max(0,node[1]-r):min(node[1]+r, shape[1]), max(0, node[2]-r):min(node[2]+r, shape[2])] = 0

    # Deal with the extra nodes
    ex_count = 0
    for ex in extra_nodes:
        node = [math.floor(n) for n in ex[0]] # get integer x, y, z
        for j in range(3):
            if node[j] > shape[j]-1:
                node[j] = shape[j]-1
        skimg[node[0], node[1], node[2]] = 0
        r = int(extra_nodes_radius[ex_count])
        zeromask[max(0,node[0]-r): min(node[0]+r, shape[0]), max(0,node[1]-r):min(node[1]+r, shape[1]), max(0, node[2]-r):min(node[2]+r, shape[2])] = 0
        ex_count += 1

    a, dm = 6, 5
    dt = skfmm.distance(skimg, dx=1)

    dt = np.exp(a * (1 - dt / dm)) - 1
    dt[zeromask == 1] = 0
    dt = (dt/np.max(dt))*255
    return dt

In [4]:
def updatePlane(swc, row, plane, axis1_coordinate, axis2_coordinate, axis1_max, axis2_max, r, p, axis1, axis2):
    plane[max(0, int(axis1_coordinate - r)):min(int(axis1_max), int(axis1_coordinate + r)), max(0, int(axis2_coordinate - r)):min(int(axis2_max), int(axis2_coordinate + r))] = 1
    for search_parent in range(0, row):
        if swc[search_parent][0] == p:
            rr, cc, val = line_aa(int(axis1_coordinate), int(axis2_coordinate), int(swc[search_parent][axis1]), int(swc[search_parent][axis2]))
            plane[rr, cc] = val * 255
            break
    return plane

In [5]:
def swc2DProjection(filepath, tif_filepath):
    img = loadtiff3d(tif_filepath)

    swc = loadswc(filepath)
    print(swc.shape)
    print(img.shape)
    x_max = np.max(swc[:, 2])
    y_max = np.max(swc[:, 3])
    z_max = np.max(swc[:, 4])
    print((x_max, y_max, z_max))
    xy_plane = np.zeros(shape=(img.shape[0], img.shape[1]))
    yz_plane = np.zeros(shape=(img.shape[1], img.shape[2]))
    xz_plane = np.zeros(shape=(img.shape[0], img.shape[2]))
    for row in range(swc.shape[0]):
        x = swc[row][2]
        y = swc[row][3]
        z = swc[row][4]
        r = swc[row][-2]
        p = swc[row][-1]
        xy_plane = updatePlane(swc, row, xy_plane, x, y, x_max, y_max, r, p, 2, 3)
        yz_plane = updatePlane(swc, row, yz_plane, y, z, y_max, z_max, r, p, 3, 4)
        xz_plane = updatePlane(swc, row, xz_plane, x, z, x_max, z_max, r, p, 2, 4)
    swc_2d_folder = filepath.rstrip('.swc') + '_swc_2D_projection/'
    if not os.path.exists(swc_2d_folder):
        os.makedirs(swc_2d_folder)

    writetiff2d(swc_2d_folder + filepath.split('.swc')[0].split('/')[-1] + '_xy.tif', (xy_plane>0)*255)
    writetiff2d(swc_2d_folder + filepath.split('.swc')[0].split('/')[-1] + '_yz.tif', (yz_plane>0)*255)
    writetiff2d(swc_2d_folder + filepath.split('.swc')[0].split('/')[-1] + '_xz.tif', (xz_plane>0)*255)

In [6]:
def projection(img):
    res = 0
    imgxy2d = img.max(axis=2)
    result = imgxy2d
    if imgxy2d.shape[0] * imgxy2d.shape[1] > res:
        res = imgxy2d.shape[0] * imgxy2d.shape[1]
    imgxz2d = img.max(axis=1)
    if imgxz2d.shape[0] * imgxz2d.shape[1] > res:
        res = imgxz2d.shape[0] * imgxz2d.shape[1]
        result = imgxz2d
    imgyz2d = img.max(axis=0)
    if imgyz2d.shape[0] * imgyz2d.shape[1] > res:
        res = imgyz2d.shape[0] * imgyz2d.shape[1]
        result = imgyz2d
        
    return result

In [13]:
path_prefix = '/media/jacktang/Work/USYD/Research/Deep_Learning/GAN/pytorch-CycleGAN-and-pix2pix/datasets/datasets/fly/fly_original/'
out_prefix = '/media/jacktang/Work/USYD/Research/Deep_Learning/GAN/pytorch-CycleGAN-and-pix2pix/datasets/datasets/fly/test_crop/'

In [16]:
import cv2
for i in range(1,43):
    img = loadtiff3d(path_prefix+str(i)+'.tif')
    img_gt = loadtiff3d(path_prefix+str(i)+'_gt.tif')
     
    x,y,z = crop(img_gt)
    img_crop = img[x[0]:x[1], y[0]:y[1], z[0]:z[1]]
    img_gt = img_gt[x[0]:x[1], y[0]:y[1], z[0]:z[1]]
    writetiff3d(out_prefix+str(i)+'.tif', img_crop)
    writetiff3d(out_prefix+str(i)+'_gt.tif', img_gt)
    img2d = projection(img_crop)
    img_gt2d = projection(img_gt)
    cv2.imwrite(out_prefix+str(i)+'_2d.tif', img2d)
    cv2.imwrite(out_prefix+str(i)+'_gt_2d.tif', img_gt2d)

In [29]:
path_prefix = '/media/jacktang/Work/USYD/Research/Deep_Learning/GAN/pytorch-CycleGAN-and-pix2pix/datasets/datasets/fly/test_crop/'
nhmax = 0
nhmin = np.inf
nwmax = 0
nwmin = np.inf
for i in range(1,43):
    gt = cv2.imread(path_prefix+str(i)+'_2d.tif',0)
#     print(gt.shape)
    h, w = gt.shape
    if h<128 or w<128:
        print(i,h,w)
    nhmax = h if h > nhmax else nhmax
    nhmin = h if h < nhmin else nhmin
    nwmax = w if w > nwmax else nwmax
    nwmin = w if w < nwmin else nwmin
    
print(nhmin, nhmax, nwmin, nwmax)

4 112 130
12 93 111
17 240 95
18 128 119
26 104 207
28 108 210
31 221 100
40 115 104
93 490 95 371


In [ ]:
path_prefix = '/media/jacktang/Work/USYD/Research/Deep_Learning/GAN/pytorch-CycleGAN-and-pix2pix/datasets/datasets/fly/fly3d/trainA/'
nhmax = 0
nhmin = np.inf
nwmax = 0
nwmin = np.inf
for i in range(1,43):
    gt = cv2.imread(path_prefix+str(i)+'i.tif',0)
#     print(gt.shape)
    h, w = gt.shape
    if h<128 or w<128:
        print(i,h,w)
    nhmax = h if h > nhmax else nhmax
    nhmin = h if h < nhmin else nhmin
    nwmax = w if w > nwmax else nwmax
    nwmin = w if w < nwmin else nwmin
    
print(nhmin, nhmax, nwmin, nwmax)

In [16]:
import subprocess
path_prefix = '/media/jacktang/Work/USYD/Research/Deep_Learning/GAN/pytorch-CycleGAN-and-pix2pix/datasets/datasets/fly/fly_original/'
for i in range(1,43):
    img = loadtiff3d(path_prefix+str(i)+'.tif')
    swc = loadswc(path_prefix+str(i)+'.swc')
    img_gt = swc2tif_dt(swc,img)
    writetiff3d(path_prefix+str(i)+'_gt.tif', img_gt)

In [24]:
import random 

ind = [x for x in range(43)]
print(ind)
path_prefix = '/media/jacktang/Work/USYD/Research/Deep_Learning/GAN/pytorch-CycleGAN-and-pix2pix/datasets/datasets/fly/test_crop/'
out_prefix = '/media/jacktang/Work/USYD/Research/Deep_Learning/GAN/pytorch-CycleGAN-and-pix2pix/datasets/datasets/fly/fly2d/'
random.shuffle(ind)
print('train: ', ind[:38])
print('val: ', ind[38:])
for i in ind[:38]:
    subprocess.call(['cp', path_prefix+str(i)+'_2d.tif', out_prefix+'trainB/'+str(i)+'_2d.tif'])
    subprocess.call(['cp', path_prefix+str(i)+'_gt_2d.tif', out_prefix+'trainA/'+str(i)+'_gt_2d.tif'])

for i in ind[38:]:
    subprocess.call(['cp', path_prefix+str(i)+'_2d.tif', out_prefix+'valA/'+str(i)+'_2d.tif'])
    subprocess.call(['cp', path_prefix+str(i)+'_gt_2d.tif', out_prefix+'valB/'+str(i)+'_gt_2d.tif'])


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42]
train:  [12, 17, 31, 38, 42, 18, 26, 4, 13, 32, 15, 5, 34, 1, 23, 29, 9, 39, 3, 11, 0, 27, 36, 41, 6, 28, 30, 19, 35, 21, 8, 2, 37, 14, 25, 7, 10, 24]
val:  [33, 16, 20, 22, 40]


In [22]:
from PIL import Image
path_prefix = '/media/jacktang/Work/USYD/Research/Deep_Learning/GAN/pytorch-CycleGAN-and-pix2pix/datasets/datasets/fly/test_crop/'
a = Image.open(path_prefix+'1.tif')

In [17]:
train = [12, 17, 31, 38, 42, 18, 26, 4, 13, 32, 15, 5, 34, 1, 23, 29, 9, 39, 3, 11, 27, 36, 41, 6, 28, 30, 19, 35, 21, 8, 2, 37, 14, 25, 7, 10, 24]
val = [33, 16, 20, 22, 40]

In [18]:
import subprocess
in_prefix = '/media/jacktang/Work/USYD/Research/Deep_Learning/GAN/pytorch-CycleGAN-and-pix2pix/datasets/datasets/fly/test_crop/'
out_prefix = '/media/jacktang/Work/USYD/Research/Deep_Learning/GAN/pytorch-CycleGAN-and-pix2pix/datasets/datasets/fly/fly3d/'

for t in train:
    subprocess.call(['cp', in_prefix+str(t)+'.tif', out_prefix+'trainB/'+str(t)+'.tif'])
    subprocess.call(['cp', in_prefix+str(t)+'_gt.tif', out_prefix+'trainA/'+str(t)+'_gt.tif'])
    
for t in val:
    subprocess.call(['cp', in_prefix+str(t)+'.tif', out_prefix+'testB/'+str(t)+'.tif'])
    subprocess.call(['cp', in_prefix+str(t)+'_gt.tif', out_prefix+'testA/'+str(t)+'_gt.tif'])

In [11]:
path_prefix1 = '/media/jacktang/Work/USYD/Research/Deep_Learning/GAN/pytorch-CycleGAN-and-pix2pix/datasets/datasets/fly/fly_original/'
path_prefix2 = '/media/jacktang/Work/USYD/Research/Deep_Learning/GAN/pytorch-CycleGAN-and-pix2pix/datasets/datasets/fly/test_crop/'

for i in range(1,43):
    a = loadtiff3d(path_prefix1+str(i)+'.tif').shape
    b = loadtiff3d(path_prefix2+str(i)+'.tif').shape
    if np.sum([a[0]-b[0],a[1]-b[1],a[2]-b[2]]) > 0:
        print(i, a, b)

In [22]:
'fake' in 'fake_b'

True